In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MLM

/content/drive/MyDrive/MLM


In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-t4pyqto2
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-t4pyqto2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 5.6MB/s 
     |████████████████████████████████| 3.2MB 9.2MB/s 
  Created wheel for transformers: filename=transformers-4.4.0.dev0-cp37-none-any.whl size=1969146 sha256=0e16cff16f8b6a28841e7581051c726ed34a02ad75bd3a15d80c19a0ee3ea5f8
  Stored in directory: /tmp/pip-ephem-wheel-cache-dal2yx29/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3ebba1f2b43e67039534a571dd0f355de796d14e201e1de99c396b9d2c8b4a55
  Stored in 

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.8MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("tlqnguyen/tiny-isbert")

model = AutoModelForMaskedLM.from_pretrained("tlqnguyen/tiny-isbert")

In [ ]:
model.num_parameters()

14381394

In [ ]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=Tr

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='tlqnguyen/tiny-isbert', vocab_size=30522, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./cxrv2.txt",
    block_size=128
)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
dataset

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./tinyisbertv2",
    overwrite_output_dir=True,
    num_train_epochs=1000,
    per_gpu_train_batch_size=128,
    save_steps=200,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [ ]:
trainer

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,2.935700
1000,2.080700
1500,1.836600
2000,1.705400
2500,1.610400
3000,1.541000
3500,1.485200
4000,1.442400
4500,1.401900
5000,1.372000
